In [46]:
import ray
import numpy as np
from ray.tune.sklearn import TuneGridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

In [50]:
ray.init(address="192.168.2.93:6379")


git_data = pd.read_csv('./githubrepos_nourls.csv')

#Include year only
git_data['created_at'] = pd.to_numeric([x.split('-')[0] for x in git_data['created_at']])

git_data['has_downloads'] = git_data['has_downloads'].astype(int)
git_data['has_issues'] = git_data['has_issues'].astype(int)
git_data['has_wiki'] = git_data['has_wiki'].astype(int)

# created_at, forks, has_downloads, has_issues, has_wiki, open_issues_count, size, watchers_count
X = git_data[['created_at', 'forks', 'has_downloads', 'has_issues', 'has_wiki', 'open_issues_count', 'size', 'subscribers_count']]
y = git_data['stargazers_count']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2)



In [39]:
#%%time

param_grid = {'bootstrap': [True, False],
              'max_depth': [20, 50, 100],
              'max_features': ['auto', 'sqrt'],
              'min_samples_leaf': [1, 2, 4],
              'min_samples_split': [2, 5, 10],
              'n_estimators': [100, 200, 400]}

rf_model = TuneGridSearchCV(RandomForestRegressor(), param_grid)
rf_model.fit(X_train, y_train)


TuneGridSearchCV(estimator=RandomForestRegressor(),
                 loggers=[<class 'ray.tune.logger.JsonLogger'>,
                          <class 'ray.tune.logger.CSVLogger'>],
                 n_jobs=-1,
                 param_grid={'bootstrap': [True, False],
                             'max_depth': [20, 50, 100],
                             'max_features': ['auto', 'sqrt'],
                             'min_samples_leaf': [1, 2, 4],
                             'min_samples_split': [2, 5, 10],
                             'n_estimators': [100, 200, 400]},
                 scoring={'score': <function _passthrough_scorer at 0x7f5aa08d55e0>},
                 sk_n_jobs=1)

In [40]:
yhat = rf_model.predict(X_test)
SS_Residual = sum((y_test-yhat)**2)       
SS_Total = sum((y-np.mean(y))**2)     
r_squared = 1 - (float(SS_Residual))/SS_Total

print("R squared for Random Forest: "+ str(r_squared))

#pickle.dump(rf_model, open("star_predict/best_model.sav", 'wb'))

R squared for Random Forest: 0.7472875762243423


In [43]:
from sklearn.model_selection import GridSearchCV

param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

#xgb_model = TuneGridSearchCV(xgb, param_grid)
xgb_model = TuneGridSearchCV(XGBRegressor(), param_grid, cv=5)
xgb_model.fit(X_train, y_train)



TuneGridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                        colsample_bylevel=None,
                                        colsample_bynode=None,
                                        colsample_bytree=None, gamma=None,
                                        gpu_id=None, importance_type='gain',
                                        interaction_constraints=None,
                                        learning_rate=None, max_delta_step=None,
                                        max_depth=None, min_child_weight=None,
                                        missing=nan, monotone_constraints=None,
                                        n_estimators=100, n_jobs=N...
                                        validate_parameters=None,
                                        verbosity=None),
                 loggers=[<class 'ray.tune.logger.JsonLogger'>,
                          <class 'ray.tune.logger.CSVLogger'>],
                 n_

In [44]:
yhat = xgb_model.predict(X_test)
SS_Residual = sum((y_test-yhat)**2)       
SS_Total = sum((y-np.mean(y))**2)     
r_squared = 1 - (float(SS_Residual))/SS_Total

print("R squared for XGBoost: "+ str(r_squared))

#pickle.dump(xgb_model, open("star_predict/xgb_model.sav", 'wb'))

R squared for XGBoost: 0.6437996171162035


In [48]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

In [49]:
yhat = lr_model.predict(X_test)
SS_Residual = sum((y_test-yhat)**2)       
SS_Total = sum((y-np.mean(y))**2)     
r_squared = 1 - (float(SS_Residual))/SS_Total

print("R squared for XGBoost: "+ str(r_squared))

#pickle.dump(xgb_model, open("star_predict/lr_model.sav", 'wb'))

R squared for XGBoost: 0.712703829720769


In [37]:
ray.shutdown()